### Random Forest Regressor

In [39]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [65]:
dj = pd.read_csv("features/dj.csv")

In [66]:
dj

,Date,High(t),Low(t),Open(t),Volume(t-5),Volume(t-4),Volume(t-3),Volume(t-2),Volume(t-1),Volume(t),Close(t+1),Close(t)
0,2002-06-13,9625.40,9491.86,9612.87,NaN,NaN,NaN,NaN,NaN,227552955,9474.21,9502.80
1,2002-06-14,9498.92,9260.99,9498.92,NaN,NaN,NaN,NaN,227552955.0,287001167,9687.42,9474.21
2,2002-06-17,9687.77,9476.50,9476.50,NaN,NaN,NaN,227552955.0,287001167.0,247554505,9706.12,9687.42
3,2002-06-18,9721.75,9636.96,9684.52,NaN,NaN,227552955.0,287001167.0,247554505.0,212707042,9561.57,9706.12
4,2002-06-19,9733.39,9542.74,9702.00,NaN,227552955.0,287001167.0,247554505.0,212707042.0,248083566,9431.77,9561.57
...,...,...,...,...,...,...,...,...,...,...,...,...
5031,2022-06-07,33207.45,32641.85,32783.03,640857432.0,344779544.0,350437139.0,327416082.0,289294618.0,327463783,32910.90,33180.14
5032,2022-06-08,33156.50,32824.37,33087.07,344779544.0,350437139.0,327416082.0,289294618.0,327463783.0,314420559,32272.79,32910.90
5033,2022-06-09,32956.73,32267.78,32828.62,350437139.0,327416082.0,289294618.0,327463783.0,314420559.0,331546530,31392.79,32272.79
5034,2022-06-10,32053.52,31387.84,32053.52,327416082.0,289294618.0,327463783.0,314420559.0,331546530.0,411949075,30516.74,31392.79


In [67]:
features_mic = pd.read_csv("./features_on_mic.csv")

In [68]:
features_mic['Close'] = dj['Close(t+1)'][2:-1]

In [70]:
features_mic = features_mic.iloc[2:-1,:]

In [71]:
features_mic

,Date,High,open_lag_1,low_lag_2,low_lag_1,high_lag_1,Low,Close_1
4,2002-06-19,9733.39,9684.52,9476.50,9636.96,9721.75,9542.74,9431.77
5,2002-06-20,9573.89,9702.00,9636.96,9542.74,9733.39,9425.88,9253.79
6,2002-06-21,9430.66,9561.64,9542.74,9425.88,9573.89,9220.63,9281.82
7,2002-06-24,9369.95,9430.66,9425.88,9220.63,9430.66,9083.56,9126.82
8,2002-06-25,9413.08,9252.47,9220.63,9083.56,9369.95,9111.25,9120.11
...,...,...,...,...,...,...,...,...
5029,2022-06-03,33135.61,32809.01,32584.76,32509.43,33248.61,32839.21,32915.78
5030,2022-06-06,33235.37,32986.32,32509.43,32839.21,33135.61,32819.50,33180.14
5031,2022-06-07,33207.45,33032.04,32839.21,32819.50,33235.37,32641.85,32910.90
5032,2022-06-08,33156.50,32783.03,32819.50,32641.85,33207.45,32824.37,32272.79


In [72]:
cols = list(features_mic.columns)[1:]
features_copy = features_mic.copy()
for col in cols:
    X = np.array(features_mic[col]).reshape((-1,1))
    SC_X = MinMaxScaler(feature_range=(0,1))
    x = SC_X.fit_transform(X).reshape((-1,))
    features_copy[col] = x

In [73]:
features_copy

,Date,High,open_lag_1,low_lag_2,low_lag_1,high_lag_1,Low,Close_1
4,2002-06-19,0.099983,0.103975,0.099667,0.104986,0.099598,0.101863,0.095354
5,2002-06-20,0.094709,0.104554,0.104986,0.101863,0.099983,0.097989,0.089471
6,2002-06-21,0.089973,0.099903,0.101863,0.097989,0.094709,0.091185,0.090398
7,2002-06-24,0.087965,0.095562,0.097989,0.091185,0.089973,0.086641,0.085274
8,2002-06-25,0.089391,0.089657,0.091185,0.086641,0.087965,0.087559,0.085053
...,...,...,...,...,...,...,...,...
5029,2022-06-03,0.873788,0.870306,0.865702,0.863205,0.877524,0.874137,0.871619
5030,2022-06-06,0.877086,0.876182,0.863205,0.874137,0.873788,0.873484,0.880357
5031,2022-06-07,0.876163,0.877697,0.874137,0.873484,0.877086,0.867595,0.871457
5032,2022-06-08,0.874479,0.869445,0.873484,0.867595,0.876163,0.873645,0.850365


In [74]:
features_copy.to_csv("features_scaled.csv")

In [31]:
len(features_mic)

5030

In [32]:
rf = RandomForestRegressor(n_estimators = 100, random_state=42)

In [33]:
X = np.array(features_mic.iloc[:,1:-1])
Y = np.array((features_mic.iloc[:,-1])).reshape((-1,1))
SC_X = MinMaxScaler(feature_range=(0,1))
SC_Y = MinMaxScaler(feature_range=(0,1))
x = SC_X.fit_transform(X)
y = SC_Y.fit_transform(Y)

Split Train Test

In [34]:
train_len = int(len(x)*0.7)
train_len

3521

In [35]:
x_train = x[:train_len,:]
x_test = x[train_len:,:]
y_train = y[:train_len].reshape(-1,)
y_test = y[train_len:].reshape(-1,)

In [41]:
param_grid = {"n_estimators":[500,800,1000],"max_depth":[8,9,10]}
grid_search = GridSearchCV(RandomForestRegressor(),param_grid,cv = 3)
grid_search.fit(x_train,y_train)
y_hat = grid_search.predict(x_test)
r2_score(y_test,y_hat)

-2.6155200274208745

In [36]:
rfModel = rf.fit(x_train,y_train)

In [37]:
y_pred_r = rfModel.predict(x_test)

In [38]:
r2_score(y_test,y_pred_r)

-2.6109117118055645

In [19]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = rfModel, X = x_train, y = y_train, cv = 10)

In [20]:
accuracies

array([0.98220153, 0.97077825, 0.97488818, 0.98015778, 0.99010857,
       0.98657172, 0.97729004, 0.99035696, 0.96846636, 0.96421015])